# Rough Notebook to be changed to pct reduction
## Last Updated: 10-30-2020

In [593]:
# Importing the necessary libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
import json
import warnings
from os import listdir
from os.path import isfile,join
import datetime
import requests
from sklearn.metrics import r2_score, accuracy_score
import catboost
import matplotlib as mpl
import sys
sys.path.append('.')
from src.utils import collect_data
from src.utils import eda


In [594]:
import importlib
importlib.reload(collect_data)
collect_datasets = collect_data.DataCollection()
data_analysis = eda.DataAnalysis()

In [595]:
df_mobility = collect_datasets.get_mobility_data(download=False)
df_county_spending = collect_datasets.get_spend_data()
df_covid = collect_datasets.combine_covid_data()
df_weather = collect_datasets.combine_weather(df_mobility,df_covid)
df_county = collect_datasets.combined_county_data()

C:\Work\projects\mobility\US_Mobility_BTS\src\utils\collect_data.py:51: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_mobility['Wk #'] = (df_mobility.date.dt.week.astype(str) ) #+ ' ' + (df_mobility.date.dt.year.astype(str) )


In [596]:
def get_df_2019(df_weather, df_mob,df_covid,df_demographics,year=2019):
    df_mob = df_mob.dropna()
    thr_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
    if year==2020:
        df_mob_prep = pd.merge(left = df_mob, right = df_covid, on = ['fips', 'date'], how='inner')
        df_mob_prep = df_mob_prep[df_mob_prep.date <thr_date]
    else:
        df_mob_prep = df_mob[(df_mob.date < thr_date)]
        
    df_mob_weather = pd.merge(left = df_mob_prep, right = df_weather, on =['date', 'fips'])

    return df_mob_weather

In [597]:
df_2019 = get_df_2019(df_weather, df_mobility, df_covid, df_demographics=df_county)

### State Mobility 2019

In [599]:
df_2019_state_mob = df_2019.groupby(['State Postal Code'])[['mobility_per_person','trips_per_person','mobility_per_person_short']].mean()
df_2019_state_mob

,mobility_per_person,trips_per_person,mobility_per_person_short
State Postal Code,,,
AK,47.061743,4.589939,20.581395
AL,45.114102,3.876168,30.996895
AR,44.884534,3.806637,29.759188
AZ,47.348724,4.047064,27.549666
CA,48.407781,4.341469,29.834772
CO,64.689916,4.837352,37.252296
CT,51.606105,3.984007,30.646906
DC,33.144677,4.970552,20.672896
DE,44.281305,3.765534,28.795667


### Fips Mobility Maps

In [621]:
cols = ['trips_per_person','mobility_per_person','mobility_per_trip_short','mobility_per_person_short',
        'pct_stay_home','mobility_per_trip']
df_2019_fil = df_2019[df_2019['State Postal Code'] != 'AK']
df_2019_fips_mob = df_2019_fil.groupby(['fips'])[cols].mean().reset_index()
fips_mob_mean_map = data_analysis.plot_map(col_list=cols,df=df_2019_fips_mob,save_op=False,cmap='plasma',q_filter=True,filter_level=[0.005,0.995])

### Weather Correlation

In [604]:
df_2019_fips_corr = df_2019_fil.groupby('fips').corr()

In [623]:
df_2019_fips_corr_ = df_2019_fips_corr.reset_index(level=1)
cols = df_mobility.columns[7:] # ['trips_per_person','mobility_per_person','pct_stay_home','mobility_per_trip']
target_cols = [col for col in df_2019_fips_corr.columns[2:17] if col not in ['Number of Trips 50-100','Number of Trips 100-250',
                                                                        'Number of Trips 250-500','Number of Trips >=500']]

df_2019_fips_corr_temp = df_2019_fips_corr_[df_2019_fips_corr_.level_1=='FeelsLikeC'][target_cols].reset_index()

feelslike_tempC_corr_map = data_analysis.plot_map(target_cols,df_2019_fips_corr_temp,cmap='bwr',q_filter=False)

In [619]:
df_2019_fips_corr_ = df_2019_fips_corr.reset_index(level=1)
target_cols = [col for col in df_2019_fips_corr.columns[2:17] if col not in ['Number of Trips 50-100','Number of Trips 100-250',
                                                                        'Number of Trips 250-500','Number of Trips >=500']]

df_2019_fips_corr_sunH = df_2019_fips_corr_[df_2019_fips_corr_.level_1=='sunHour'][target_cols].reset_index()

sunhour_tempC_corr_map = data_analysis.plot_map(target_cols,df_2019_fips_corr_sunH,cmap='bwr',q_filter=True, filter_level=[0.005,0.995])

### Time Series

In [618]:
targ_var = 'mobility_per_person'
ts_base_map = data_analysis.plot_ts_map(df_2019_fil,targ_var,q_filter=True)

C:\Work\projects\mobility\US_Mobility_BTS\src\utils\eda.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].astype(int)


### Weekend Percentage Changes in Mobility

In [616]:
target_cols = df_mobility.columns[7:] # ['trips_per_person','mobility_per_person','pct_stay_home','mobility_per_trip']
# target_cols = [col for col in cols if col not in ['Number of Trips 50-100','Number of Trips 100-250','pct_not_home',
#                                                                         'Number of Trips 250-500','Number of Trips >=500']]
df_2019_fips_weekday = df_2019_fil.groupby(['fips','weekend'])[target_cols].mean().reset_index()
df_2019_fips_weekday

,fips,weekend,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500,mobility_per_trip,mobility_per_person,mobility_per_trip_short,mobility_per_person_short,pct_stay_home,trips_per_person,pct_not_home
0,01001,Weekday,39774.555556,59619.003831,37112.934866,38371.007663,44097.394636,10257.386973,3194.819923,1197.406130,244.141762,43.490421,9.953082,41.584033,7.416319,31.240930,14.224132,4.219880,85.775868
1,01001,Weekend,39322.846154,63145.221154,39224.125000,39934.855769,36240.740385,9049.980769,3079.750000,1304.221154,270.096154,42.480769,9.474846,39.117346,6.816027,28.430788,15.655968,4.178426,84.344032
2,01003,Weekday,143486.551724,179656.950192,100129.187739,138346.593870,147413.402299,50204.126437,9825.337165,3295.532567,1398.187739,391.850575,11.006441,38.937915,8.219027,29.129919,15.846785,3.561658,84.153215
3,01003,Weekend,138393.105769,180230.980769,98495.230769,132602.182692,129024.442308,43351.259615,11092.105769,3989.250000,1703.307692,425.701923,11.142076,37.679196,7.737477,26.232337,16.925844,3.401367,83.074156
4,01005,Weekday,17159.126437,21119.321839,12993.754789,16375.551724,15414.406130,8569.934866,2960.808429,523.662835,112.172414,56.992337,12.821022,48.991940,8.592724,32.926206,14.788422,3.841392,85.211578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,56041,Weekend,23158.634615,28257.528846,10815.480769,5776.807692,9227.913462,4258.913462,6370.865385,1876.471154,192.307692,489.250000,19.268738,85.048690,5.255526,23.515348,21.351504,4.468260,78.648496
6218,56043,Weekday,14638.609195,6283.804598,4170.022989,2083.708812,2381.272031,4488.804598,1401.636015,567.318008,125.704981,32.808429,14.941000,68.483182,7.123268,33.354186,39.404773,4.601665,60.595227
6219,56043,Weekend,12873.471154,5841.432692,4329.480769,2165.038462,2586.048077,3911.894231,1496.451923,646.028846,95.394231,33.048077,15.866772,67.354665,7.082624,30.991882,40.216429,4.322388,59.783571
6220,56045,Weekday,5770.279693,4780.754789,1139.528736,679.693487,4128.927203,2844.478927,12315.961686,336.000000,87.122605,64.000000,39.297280,182.003276,6.309914,28.941355,18.392846,4.628095,81.607154


In [ ]:
cols = [col for col in df_2019_fips_weekday.columns if col not in ['fips','weekend']]

df_2019_fips_weekday_pct_change = (df_2019_fips_weekday.groupby('fips')[cols].apply(pd.Series.pct_change)) *100
df_2019_fips_weekday_pct_change['fips'] = df_2019_fips_weekday['fips']

df_2019_fips_weekday_pct_change = df_2019_fips_weekday_pct_change.dropna()
pct_change_weekend_map = data_analysis.plot_map(cols,df_2019_fips_weekday_pct_change,cmap='bwr',q_filter=True,filter_level=[0.005, 0.995])

In [ ]:
targ_var = 'trips_per_person'
trips_per_person_ts_map = data_analysis.plot_ts_map(df_2019_fil,targ_var,q_filter=True)